In [1]:
import pandas as pd
import itertools

The file below contains the predictions NER tag for sentences in train, testa, testb data from conll dataset, caluclated based on the model made by me 

In [2]:
with open('entire.output') as fl:
    output = fl.read().split("\n")

In [3]:
df = pd.DataFrame(columns=['Tokens', 'NERTag', 'WikiPage'])

This file contains the Wiki Page links to entities/ sequence of tokens/ sequence of words in Conll dataset

In [4]:
with open('AIDA-YAGO2-dataset.tsv') as fl:
    aida = fl.read().split("\n")

In [5]:
output_final=[]
# Removing blank lines, DOCTSTART lines from both files 
for k in range(0, len(output)):
    
    if "DOCSTART" in output[k] or output[k] is "" :
        continue
    output_final.append(output[k])
    
aida_final=[]
for m in range(0,len(aida)):
    if "DOCSTART" in aida[m] or aida[m] is "" :
        continue
    aida_final.append(aida[m])

In [6]:
len(output_final),len(aida_final)

(301418, 301418)

I am using these two files to calculate the probability of Wiki pages. As seen below the variable output_final tells us the predicted NER Tag in the second column for each word. 

The aida_final variable tells us the Wiki Page for certain words/Sequence of words. Each word with Tag(second column) as 'B' is an entity whose Wiki Page might be given. Each word with Tag(second column) as 'I' is continuation of previous word like 'European Commission'.

So i mapped the output_final, aida_final word by word to get NER tag of all word/sequence of words whose Wiki Page is available

In [7]:
output_final[0:9], aida_final[0:9]

(['EU I-ORG I-ORG',
  'rejects O O',
  'German I-MISC I-MISC',
  'call O O',
  'to O O',
  'boycott O O',
  'British I-MISC I-MISC',
  'lamb O O',
  '. O O'],
 ['EU\tB\tEU\t--NME--',
  'rejects',
  'German\tB\tGerman\tGermany\thttp://en.wikipedia.org/wiki/Germany\t11867\t/m/0345h',
  'call',
  'to',
  'boycott',
  'British\tB\tBritish\tUnited_Kingdom\thttp://en.wikipedia.org/wiki/United_Kingdom\t31717\t/m/07ssc',
  'lamb',
  '.'])

In [8]:

Tokens=""
Url=""
for i in range(0,len(aida_final)):
    
    f=output_final[i]
    b=aida_final[i]
    f=f.split(" ")
   
    b=b.split("\t")
    if(f[0] !=b[0]):
        print("something wrong",f,b)
        break
    
    if(len(b)>4):
        if(b[4]!=""):
            if(b[1]=='B'):
                
                
                Tokens= b[2]
                Url=b[4]
                Tag=f[1]
               


               
                df = df.append({
                 "Tokens": Tokens,
                 "NERTag":  Tag,
                  "WikiPage":Url
                  }, ignore_index=True)
    
    

In [9]:
df.head(10)

,Tokens,NERTag,WikiPage
0,German,I-MISC,http://en.wikipedia.org/wiki/Germany
1,British,I-MISC,http://en.wikipedia.org/wiki/United_Kingdom
2,BRUSSELS,I-LOC,http://en.wikipedia.org/wiki/Brussels
3,European Commission,I-ORG,http://en.wikipedia.org/wiki/European_Commission
4,German,I-MISC,http://en.wikipedia.org/wiki/Germany
5,British,I-MISC,http://en.wikipedia.org/wiki/United_Kingdom
6,Germany,I-LOC,http://en.wikipedia.org/wiki/Germany
7,European Union,I-ORG,http://en.wikipedia.org/wiki/European_Union
8,Britain,I-LOC,http://en.wikipedia.org/wiki/United_Kingdom
9,Commission,I-ORG,http://en.wikipedia.org/wiki/European_Commission


In [10]:

groupby_page=df.groupby(['Tokens','NERTag','WikiPage'])


In [11]:
groupby_key=df.groupby(['Tokens','NERTag']).size()


In [12]:
df_prob = pd.DataFrame(columns=['Tokens', 'NERTag', 'WikiPage','Probability' ])
for name, group in groupby_page:
    
    p=len(group)/groupby_key[name[0],name[1]]
    
    df_prob = df_prob.append({
                 "Tokens": name[0],
                 "NERTag":  name[1],
                 "WikiPage":name[2],
                 "Probability":p
                  }, ignore_index=True)
    
    

In [13]:
df_prob=df_prob.groupby('Tokens').apply(lambda x: x.sort_values('Probability', ascending=False))

In [14]:
df_prob.loc['German']

,Tokens,NERTag,WikiPage,Probability
2654,German,I-ORG,http://en.wikipedia.org/wiki/Germany,1.000000
2652,German,I-MISC,http://en.wikipedia.org/wiki/Germany,0.946903
2650,German,I-MISC,http://en.wikipedia.org/wiki/German_language,0.026549
2653,German,I-MISC,http://en.wikipedia.org/wiki/Nazi_Germany,0.017699
2651,German,I-MISC,http://en.wikipedia.org/wiki/Germans,0.008850


In [15]:
df_prob.loc['Germany']

,Tokens,NERTag,WikiPage,Probability
2660,Germany,I-LOC,http://en.wikipedia.org/wiki/Germany,0.961864
2664,Germany,I-LOC,http://en.wikipedia.org/wiki/Germany_national_...,0.025424
2661,Germany,I-LOC,http://en.wikipedia.org/wiki/Germany_Davis_Cup...,0.004237
2662,Germany,I-LOC,http://en.wikipedia.org/wiki/Germany_Fed_Cup_team,0.004237
2663,Germany,I-LOC,http://en.wikipedia.org/wiki/Germany_men's_nat...,0.004237


In [16]:
df_prob.loc['British']

,Tokens,NERTag,WikiPage,Probability
1075,British,I-MISC,http://en.wikipedia.org/wiki/United_Kingdom,0.989247
1074,British,I-MISC,http://en.wikipedia.org/wiki/England,0.010753


In [17]:
df_prob.loc['Costantino Rocca']

,Tokens,NERTag,WikiPage,Probability
1626,Costantino Rocca,I-PER,http://en.wikipedia.org/wiki/Costantino_Rocca,1.0


Now we assume that for each token the Wiki Page linked to it will be the one with highest probability. Now the accuracy can be calculating the number of times that this max probability page is not mapped to the token in the aida data

In [18]:
correct, wrong = 0, 0
for index, row in df.iterrows():
    ent = row["Tokens"]
    tag = row["NERTag"]
    actual_link = row["WikiPage"]
    predicted_link = df_prob.loc[df_prob['NERTag'] == tag].loc[ent].iloc[0].loc['WikiPage']
    
    if actual_link == predicted_link:
        correct = correct +1
    else:
        wrong = wrong +1
        
correct, wrong


(26286, 1531)

# Accuracy

In [19]:
acc = 100. * correct/(correct+wrong)
acc

94.49617140597476